In [2]:
import os
print(os.getcwd())
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

/home/sepidehparhami/tutoring/Research Project


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten

In [4]:
yields = pd.read_csv(r'yield.txt', delim_whitespace=True, index_col=0)

In [5]:
ligands = list(yields.index)

In [6]:
sample_file = pd.read_csv('./data/ESP allPoints_L01-DPPF.txt_interpolated.txt')
sample_file.columns = ('x', 'y', 'z', 'esp')

In [24]:
print(len(np.unique(sample_file['x'])),len(np.unique(sample_file['y'])),len(np.unique(sample_file['z'])))

30 30 30


In [31]:
np.max(sample_file['x'])

14.0

In [18]:
sample_file.shape

(26999, 4)

In [7]:
n_molecules = yields.shape[0]
n_coords = sample_file.shape[0]
dim_x, dim_y, dim_z = (len(np.unique(sample_file[data_axis])) for data_axis in ['x','y','z'])

In [44]:
# initialize a data frame and fill in ESPs from file

data = pd.DataFrame(index = ligands, columns=range(n_coords+1))

In [45]:
directory = './data' 
      
for i,filename in enumerate(os.listdir(directory)):
    # print(i+1,'of',n_molecules)

    data_point = pd.read_csv('./data/{}'.format(filename))
    data_point.columns = ('x', 'y', 'z', 'esp')
    
    # print('number coords in file:',data_point.shape[0])

    #extract the ligand name from the filename
    name_parts = filename.split("_")
    if name_parts[1][0] == "T":
        ligand = name_parts[1][:4]
    else:
        ligand = name_parts[1][:3]
        
    #add label and data point to their respective arrays 
    data.loc[ligand] = data_point['esp']

In [54]:
data[n_coords] = np.zeros(n_molecules,dtype=type(data.iloc[0,0]))

In [55]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,26990,26991,26992,26993,26994,26995,26996,26997,26998,26999
L01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
# turn data frame into 5-dimensional array (n_molecules,x,y,z,1 (size of ESP value)) array for keras input 
data_arr_5d = np.zeros(n_molecules,dtype=object)
for i,l in enumerate(ligands):
    data_arr_5d[i] = np.reshape(data.loc[l].values, (dim_x,dim_y,dim_z,1))
# print(data_arr_5d[0])
print(data_arr_5d.shape)

(39,)


In [61]:
data_arr_5d[i]

array([[[[0.0],
         [0.0],
         [0.0],
         ...,
         [0.0],
         [0.0],
         [0.0]],

        [[0.0],
         [0.0],
         [0.0],
         ...,
         [0.0],
         [0.0],
         [0.0]],

        [[0.0],
         [0.0],
         [0.0],
         ...,
         [0.0],
         [0.0],
         [0.0]],

        ...,

        [[0.0],
         [0.0],
         [0.0],
         ...,
         [0.0],
         [0.0],
         [0.0]],

        [[0.0],
         [0.0],
         [0.0],
         ...,
         [0.0],
         [0.0],
         [0.0]],

        [[0.0],
         [0.0],
         [0.0],
         ...,
         [0.0],
         [0.0],
         [0.0]]],


       [[[0.0],
         [0.0],
         [0.0],
         ...,
         [0.0],
         [0.0],
         [0.0]],

        [[0.0],
         [0.0],
         [0.0],
         ...,
         [0.0],
         [0.0],
         [0.0]],

        [[0.0],
         [0.0],
         [0.0],
         ...,
         [0.0],
         [

In [32]:
sample_file.iloc[-1]

x      13.0
y      14.0
z      14.0
esp     0.0
Name: 26998, dtype: float64

In [62]:
model = Sequential()
model.add(Conv3D(n_molecules,kernel_size=10,activation='relu',input_shape=(dim_x,dim_y,dim_z,1)))
model.add(Flatten())
model.add(Dense(3,activation='softmax'))